In [7]:
##################
# Libraries
##################
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from loguru import logger

In [2]:
##################
# Constants
##################
MODEL_SEED=42
ROWS_SEED=[24, 42, 206, 602, 412, 214, 754, 457, 2023, 3202]
SIZE_OF_UNLABELLED_DATA = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9]
MODELS = [
    GaussianNB(var_smoothing=0.657933224657568),
    DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=6, min_samples_split=7),
    KNeighborsClassifier(metric='euclidean', weights='distance'),
    RandomForestClassifier(criterion='gini', max_depth=15, min_samples_leaf=2, min_samples_split=2, n_estimators=100, ),
]

In [10]:
##################
# Functions
##################
def encode_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Encode dataframe using LabelEncoder"""
    labelencoder = LabelEncoder()

    for col in df.columns:
        df[col] = labelencoder.fit_transform(df[col])

    return df

def get_metrics() -> dict:
    metrics = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'f1': f1_score,
        'roc': roc_auc_score
    }
    return metrics


def get_metrics_df(y_true, y_pred) -> pd.DataFrame:
    metrics = get_metrics()
    df = pd.DataFrame()
    for metric_name, metric in metrics.items():
        df[metric_name] = [metric(y_true, y_pred)]

    return df

def run_pipeline(X_train, X_test, y_train, y_test, size_of_unlabelled_data=0.5, model=None, rows_seed=42) -> dict:
    if model is None:
        raise ValueError("Model is None")
    
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_train = y_train.copy()
    y_test = y_test.copy()

    # Unlabelled data
    rng = np.random.RandomState(rows_seed)
    random_rows_with_rng = rng.choice(X_train.index, size=int(len(X_train)*size_of_unlabelled_data), replace=False)
    y_train.loc[random_rows_with_rng] = -1

    # Scale data
    scaler = RobustScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


    self_training_model = SelfTrainingClassifier(model, verbose=False)
    self_training_model.fit(X_train, y_train)

    # Predict
    y_pred = self_training_model.predict(X_test)

    # Evaluate - Get Metrics
    metrics = get_metrics_df(y_test, y_pred)
    accuracy = metrics["accuracy"].values[0]
    precision = metrics["precision"].values[0]
    recall = metrics["recall"].values[0]
    f1 = metrics["f1"].values[0]
    roc = metrics["roc"].values[0]
    

    return {"rows_seed": rows_seed, "model": type(model).__name__, "unlabeled": size_of_unlabelled_data, "acc": accuracy, "precision": precision, "recall": recall, "f1": f1, "roc": roc}

def run_pipeline_self(X_train, X_test, y_train, y_test) -> pd.DataFrame:
    run_counter = 0
    results = []

    for model in MODELS:
        for value in SIZE_OF_UNLABELLED_DATA:
            logger.info(f'Model: {model}')
            logger.info(f'Size of unlabelled data: {value}')
            for row_seed in ROWS_SEED:
                run_counter += 1
                logger.info(f"%{run_counter} - Running pipeline for row_seed: {row_seed} and size_of_unlabelled_data: {value} and model: {model}")
                result = run_pipeline(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, size_of_unlabelled_data=value, model=model, rows_seed=row_seed)
                results.append(result)
            print('\n')

    results_rows_seed = []
    results_models = []
    results_unlabeled = []
    results_acc = []
    results_precision = []
    results_recall = []
    results_f1 = []
    results_roc = []

    for result in results:
        results_rows_seed.append(result.get('rows_seed'))
        results_models.append(result.get('model'))
        results_unlabeled.append(result.get('unlabeled'))
        results_acc.append(result.get('acc'))
        results_precision.append(result.get('precision'))
        results_recall.append(result.get('recall'))
        results_f1.append(result.get('f1'))
        results_roc.append(result.get('roc'))


    results_df = pd.DataFrame({'rows_seed': results_rows_seed, 'model': results_models, 'unlabeled': results_unlabeled, 'acc': results_acc, 'precision': results_precision, 'recall': results_recall, 'f1': results_f1, 'roc': results_roc})

    return results_df

    

In [4]:
df = pd.read_csv('../../datasets/diabetes.csv')
df = encode_dataframe(df)


X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=MODEL_SEED)

In [11]:
results_df = run_pipeline_self(X_train, X_test, y_train, y_test)

2023-10-22 10:50:46.789 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:46.790 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.1
2023-10-22 10:50:46.791 | INFO     | __main__:run_pipeline_self:79 - %1 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:46.803 | INFO     | __main__:run_pipeline_self:79 - %2 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:46.816 | INFO     | __main__:run_pipeline_self:79 - %3 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:46.828 | INFO     | __main__:run_pipeline_self:79 - %4 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.6579332246575

2023-10-22 10:50:47.122 | INFO     | __main__:run_pipeline_self:79 - %27 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.135 | INFO     | __main__:run_pipeline_self:79 - %28 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.148 | INFO     | __main__:run_pipeline_self:79 - %29 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.161 | INFO     | __main__:run_pipeline_self:79 - %30 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.175 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.176 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelle

2023-10-22 10:50:47.383 | INFO     | __main__:run_pipeline_self:79 - %44 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.410 | INFO     | __main__:run_pipeline_self:79 - %45 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.432 | INFO     | __main__:run_pipeline_self:79 - %46 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.446 | INFO     | __main__:run_pipeline_self:79 - %47 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.460 | INFO     | __main__:run_pipeline_self:79 - %48 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22

2023-10-22 10:50:47.718 | INFO     | __main__:run_pipeline_self:79 - %65 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.737 | INFO     | __main__:run_pipeline_self:79 - %66 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.756 | INFO     | __main__:run_pipeline_self:79 - %67 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.774 | INFO     | __main__:run_pipeline_self:79 - %68 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:47.789 | INFO     | __main__:run_pipeline_self:79 - %69 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-2

2023-10-22 10:50:48.026 | INFO     | __main__:run_pipeline_self:79 - %86 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:48.042 | INFO     | __main__:run_pipeline_self:79 - %87 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:48.057 | INFO     | __main__:run_pipeline_self:79 - %88 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:48.075 | INFO     | __main__:run_pipeline_self:79 - %89 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.657933224657568)
2023-10-22 10:50:48.089 | INFO     | __main__:run_pipeline_self:79 - %90 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.657933224657568)
202

...

2023-10-22 10:51:33.853 | INFO     | __main__:run_pipeline_self:79 - %352 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:34.806 | INFO     | __main__:run_pipeline_self:79 - %353 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:35.766 | INFO     | __main__:run_pipeline_self:79 - %354 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:36.721 | INFO     | __main__:run_pipeline_self:79 - %355 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:37.694 | INFO     | __main__:run_pipeline_self:79 - %356 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.6 and model

2023-10-22 10:51:43.211 | INFO     | __main__:run_pipeline_self:79 - %362 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:44.157 | INFO     | __main__:run_pipeline_self:79 - %363 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:45.103 | INFO     | __main__:run_pipeline_self:79 - %364 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:45.873 | INFO     | __main__:run_pipeline_self:79 - %365 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:46.815 | INFO     | __main__:run_pipeline_self:79 - %366 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.7 and model

2023-10-22 10:51:52.483 | INFO     | __main__:run_pipeline_self:79 - %372 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:53.404 | INFO     | __main__:run_pipeline_self:79 - %373 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:54.324 | INFO     | __main__:run_pipeline_self:79 - %374 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:55.256 | INFO     | __main__:run_pipeline_self:79 - %375 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:51:56.197 | INFO     | __main__:run_pipeline_self:79 - %376 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.8 and model

2023-10-22 10:52:01.631 | INFO     | __main__:run_pipeline_self:79 - %382 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:52:02.529 | INFO     | __main__:run_pipeline_self:79 - %383 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:52:03.444 | INFO     | __main__:run_pipeline_self:79 - %384 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:52:04.385 | INFO     | __main__:run_pipeline_self:79 - %385 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:52:05.304 | INFO     | __main__:run_pipeline_self:79 - %386 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.85 and 

2023-10-22 10:52:10.873 | INFO     | __main__:run_pipeline_self:79 - %392 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:52:11.788 | INFO     | __main__:run_pipeline_self:79 - %393 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:52:12.696 | INFO     | __main__:run_pipeline_self:79 - %394 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:52:13.596 | INFO     | __main__:run_pipeline_self:79 - %395 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_leaf=2)
2023-10-22 10:52:14.533 | INFO     | __main__:run_pipeline_self:79 - %396 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.9 and model

In [12]:
# mean and standard deviation for all metrics
results_df_with_mean_std = results_df.groupby(['model', 'unlabeled']).agg({'acc': ['mean', 'std'], 'f1': ['mean', 'std'], 'precision': ['mean', 'std'], 'recall': ['mean', 'std'], 'roc': ['mean', 'std']})
results_df_with_mean_std = results_df_with_mean_std.reset_index(level=1)
results_df_with_mean_std.columns = ["_".join(col) for col in results_df_with_mean_std.columns.values]
results_df_with_mean_std.rename(columns={'unlabeled_': 'unlabeled'}, inplace=True)
results_df_with_mean_std.reset_index(inplace=True)
results_df_with_mean_std.columns

Index(['model', 'unlabeled', 'acc_mean', 'acc_std', 'f1_mean', 'f1_std',
       'precision_mean', 'precision_std', 'recall_mean', 'recall_std',
       'roc_mean', 'roc_std'],
      dtype='object')

In [8]:
import plotly.express as px

fig1 = px.scatter(results_df_with_mean_std, x='unlabeled', y='acc_mean', error_y='acc_std', color='model', title='Accuracy vs. Unlabeled Data')
fig2 = px.scatter(results_df_with_mean_std, x='unlabeled', y='f1_mean', error_y='f1_std', color='model', title='F1 Score vs. Unlabeled Data')
fig3 = px.scatter(results_df_with_mean_std, x='unlabeled', y='precision_mean', error_y='precision_std', color='model', title='Precision vs. Unlabeled Data')
fig4 = px.scatter(results_df_with_mean_std, x='unlabeled', y='recall_mean', error_y='recall_std', color='model', title='Recall vs. Unlabeled Data')
fig5 = px.scatter(results_df_with_mean_std, x='unlabeled', y='roc_mean', error_y='roc_std', color='model', title='ROC AUC vs. Unlabeled Data')


figs = [fig1, fig2, fig3, fig4, fig5]

for fig in figs:
    fig.update_layout(
        xaxis_title="% of Unlabeled Data in Dataset",
        yaxis_title="Metric Value",
        legend_title="Model",
        autosize=False,
        width=1200,
        height=500,
    )
    fig.update_traces(textposition='top center', texttemplate='%{y:.3f}')
    fig.update_layout(hovermode="x unified")
    fig.show()


In [15]:
# Metrics with tabulate
from tabulate import tabulate

print(tabulate(results_df_with_mean_std, headers='keys', tablefmt='psql', showindex=False))

+------------------------+-------------+------------+------------+-----------+-----------+------------------+-----------------+---------------+--------------+------------+------------+
| model                  |   unlabeled |   acc_mean |    acc_std |   f1_mean |    f1_std |   precision_mean |   precision_std |   recall_mean |   recall_std |   roc_mean |    roc_std |
|------------------------+-------------+------------+------------+-----------+-----------+------------------+-----------------+---------------+--------------+------------+------------|
| DecisionTreeClassifier |        0.1  |   0.732468 | 0.0310363  |  0.612157 | 0.0593319 |         0.635283 |       0.0519972 |      0.598182 |    0.0963808 |   0.702626 | 0.0399787  |
| DecisionTreeClassifier |        0.2  |   0.722727 | 0.0181225  |  0.612012 | 0.0442131 |         0.610771 |       0.0217363 |      0.618182 |    0.0808586 |   0.699495 | 0.0295079  |
| DecisionTreeClassifier |        0.3  |   0.708442 | 0.0367902  |  0.58444